In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import pinv2
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read Data
data = pd.read_csv("aapl.csv")
#Display top 10 rows
data.head(10)

,Date,Open,High,Low,Close,Volume
0,1/3/2006,6.999730,7.228929,6.987158,7.228929,201808600
1,1/4/2006,7.265678,7.347880,7.204753,7.250205,154900900
2,1/5/2006,7.236664,7.243433,7.132219,7.193145,112355600
3,1/6/2006,7.277282,7.417508,7.209586,7.378825,176114400
4,1/9/2006,7.420411,7.465863,7.324670,7.354649,168760200
5,1/10/2006,7.373992,7.919425,7.333374,7.819816,569967300
6,1/11/2006,8.108005,8.200844,7.987120,8.113807,373448600
7,1/12/2006,8.217287,8.355579,8.086730,8.151525,320202400
8,1/13/2006,8.219220,8.317862,8.181504,8.277245,194076400
9,1/17/2006,8.287883,8.353644,8.110907,8.192142,208905900


# Indicator Functions

In [3]:
# Indicator functions
def MA(data,period):
    
    '''
    Caluclates the moving averages for a given period
    '''
    
    SMA = data['Close'].rolling(window = period).mean()
#     SMA = SMA.shift(periods = -(period-1))
    SMA = pd.Series(SMA, name = 'SMA') 
    data = data.join(SMA) 
    return data

def EMA(data,period):
    '''
    Caluclates the exponential moving averages for a given period
    '''
    EMA = pd.Series(data['Close'].ewm(span = period, min_periods = period - 1).mean(), 
                 name = 'EWMA_' + str(period)) 
    data = data.join(EMA) 
    return data

def ROC(data, period):
    '''
    Caluclates the Rate of Change for a given period
    '''
    N = data['Close'].diff(periods = period)
    D = data['Close'].shift(periods = period)
    ROC = pd.Series(N/D,name='ROC')
    data = data.join(ROC)
    return data

def RSI(data,period):
    '''
    Calculates the Relative Strength Index for a given period
    '''
    dif = data['Close'].diff()
    dUp, dDown = dif.copy(), dif.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(window = period).mean()
    RolDown = dDown.abs().rolling(window = period).mean()

    RSI_1 = RolUp / RolDown
    RSI_2= 100.0 - (100.0 / (1.0 + RSI_1))
    RSI = pd.Series(RSI_2, name='RSI')
    data = data.join(RSI) 
    return data

def Stoch_Osc(data, period, percent_k_slow_period, percent_d_period):
    '''
    Calculates the Stochastic Oscillator for the given period
    using the %slow_period and %dperiod for SLOWK and SLOWD
    '''
    
    low_min = data['Low'].rolling(window=period).min()
    
    high_max = data['High'].rolling(window=period).max()
    low_min_avg = low_min.rolling(window=percent_k_slow_period).mean()
    high_max_avg = high_max.rolling(window=percent_k_slow_period).mean()
    
    FAST_percent_K = 100*((data['Close'] - low_min) / (high_max - low_min))
    FASTK = pd.Series(FAST_percent_K, name='FASTK')
    SLOW_percent_K = 100*((data['Close'] - low_min_avg) / (high_max_avg - low_min_avg))
    SLOWK = pd.Series(SLOW_percent_K, name='SLOWK')
    
    SLOW_percent_D= SLOW_percent_K.rolling(window=percent_d_period).mean()
    SLOWD = pd.Series(SLOW_percent_D, name='SLOWD')
    data = data.join(FASTK)
    data = data.join(SLOWK)
    data = data.join(SLOWD)
    return data

def boundary(data, n1):
    '''
    Create boundaries:
    Upk = max(Ci+1,Ci+2, . . . ,Ci+n1 )
    Lowk = min(Ci+1,Ci+2, . . . ,Ci+n1 )
    '''
    min_vals = data['Close'].rolling(window = n1).min()
    Downk = min_vals.shift(periods = -(n1-1))
    max_vals = data['Close'].rolling(window = n1).max()
    Upk = max_vals.shift(periods = -(n1-1))
    Downk = pd.Series(Downk, name='DOWN_K')
    data = data.join(Downk)
    Upk = pd.Series(Upk, name='UP_K')
    data = data.join(Upk)
    return data

In [4]:
def data_w_indicators(n1,period, percent_k_slow_period, percent_d_period):
    data = pd.read_csv("aapl.csv")
#     data = MA(data,period)
    data = EMA(data,period)
    data = ROC(data,period)
    data = RSI(data,period)
    data = Stoch_Osc(data,period,percent_k_slow_period,percent_d_period)
    data = boundary(data, n1)
    return data

In [5]:
data = data_w_indicators(n1= 15, period = 15, percent_k_slow_period = 3, percent_d_period= 3)
data.head()

,Date,Open,High,Low,Close,Volume,EWMA_15,ROC,RSI,FASTK,SLOWK,SLOWD,DOWN_K,UP_K
0,1/3/2006,6.999730,7.228929,6.987158,7.228929,201808600,NaN,NaN,NaN,NaN,NaN,NaN,7.193145,8.277245
1,1/4/2006,7.265678,7.347880,7.204753,7.250205,154900900,NaN,NaN,NaN,NaN,NaN,NaN,7.175739,8.277245
2,1/5/2006,7.236664,7.243433,7.132219,7.193145,112355600,NaN,NaN,NaN,NaN,NaN,NaN,6.994895,8.277245
3,1/6/2006,7.277282,7.417508,7.209586,7.378825,176114400,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245
4,1/9/2006,7.420411,7.465863,7.324670,7.354649,168760200,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245


In [6]:
model_data = data.drop(['Date'],axis=1)
model_data.head()

,Open,High,Low,Close,Volume,EWMA_15,ROC,RSI,FASTK,SLOWK,SLOWD,DOWN_K,UP_K
0,6.999730,7.228929,6.987158,7.228929,201808600,NaN,NaN,NaN,NaN,NaN,NaN,7.193145,8.277245
1,7.265678,7.347880,7.204753,7.250205,154900900,NaN,NaN,NaN,NaN,NaN,NaN,7.175739,8.277245
2,7.236664,7.243433,7.132219,7.193145,112355600,NaN,NaN,NaN,NaN,NaN,NaN,6.994895,8.277245
3,7.277282,7.417508,7.209586,7.378825,176114400,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245
4,7.420411,7.465863,7.324670,7.354649,168760200,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(model_data)
X = scaler.transform(model_data)
X = pd.DataFrame(X, columns = model_data.columns)
X.head()

,Open,High,Low,Close,Volume,EWMA_15,ROC,RSI,FASTK,SLOWK,SLOWD,DOWN_K,UP_K
0,-0.977509,-0.976298,-0.975635,-0.973660,-0.542341,NaN,NaN,NaN,NaN,NaN,NaN,-0.973040,-0.970749
1,-0.974514,-0.974964,-0.973154,-0.973419,-0.655132,NaN,NaN,NaN,NaN,NaN,NaN,-0.973245,-0.970749
2,-0.974841,-0.976135,-0.973981,-0.974065,-0.757433,NaN,NaN,NaN,NaN,NaN,NaN,-0.975371,-0.970749
3,-0.974384,-0.974184,-0.973099,-0.971964,-0.604123,NaN,NaN,NaN,NaN,NaN,NaN,-0.975712,-0.970749
4,-0.972772,-0.973642,-0.971786,-0.972238,-0.621807,NaN,NaN,NaN,NaN,NaN,NaN,-0.975712,-0.970749


# Extreme Machine Learning

In [8]:
def feature_matrix(X,i,n2):
    return X.iloc[i-n2:i,:]

In [9]:
def Gray_correlation(X,t):
    n1 = np.nanmin((np.abs(np.asarray(X) - np.asarray(t))),axis = 0)
    n2 = 0.5*np.nanmax((np.abs(np.asarray(X) - np.asarray(t))),axis = 0)
    d1 = (np.abs(np.asarray(X) - np.asarray(t)))
    d2 = 0.5*np.nanmax((np.abs(np.asarray(X) - np.asarray(t))),axis = 0)
    wi = (n1+n2)/(d1+d2)
    wi[np.isnan(wi)]=1
    return wi

In [10]:
def Hadamard(X,Y):
    return X*Y

In [11]:
def target_data(X,i,n1):
    return X.loc[i+1:i+n1,['Close']].values

In [12]:
def training_data_generation(X, i, n, n1, n2):
    '''
    Arguments:
    X -- input dataset
    i -- ith day after which the stock price needs to be predicted
    n -- window size
    n1 -- n1 days of data to be predict
    n2 -- n2 days of history data
    
    Returns:
    I -- input weight matrix [Ii-n, Ii-n+1, ..., Ii-n1]
    T -- target matrix [Ti−n, Ti−n+1, . . . , Ti−n1]
    
    '''
    I = np.asarray([])
    T = np.asarray([])
    for j in range(i-n, i-n1+1):
        F_t = feature_matrix(X,j,n2)
        W_t = Gray_correlation(F_t,F_t.loc[:,['Close']])
        I_t = Hadamard(np.asarray(F_t),W_t)
        I_t = I_t.flatten()
        I_t = I_t.reshape(I_t.shape[0],1)
        I = np.hstack((I, I_t)) if I.size else I_t
        t_i = target_data(X,j,n1)
        T = np.hstack((T, t_i)) if T.size else t_i
    return I, T

In [13]:
def test_data_generation(X, i, n1, n2):
    F_val = feature_matrix(X,i,n2)
    W_val = Gray_correlation(F_val, F_val.loc[:,['Close']])
    I_val = Hadamard(np.asarray(F_val), W_val)
    I_val = I_val.flatten()
    I_val = I_val.reshape(I_val.shape[0],1)
    T_val = target_data(X,i,n1)
    return I_val, T_val
    

In [14]:
def sigmoid(z):
    """
    return the sigmoid of z
    """    
    return 1/ (1 + np.exp(-z))

In [15]:
def rbf(z):
    """
    return the rbf of z
    """    
    return np.exp(-(z*z))

In [16]:
def triang_basis(z):
    """
    return the triangular basis of z
    """    
    if z>=-1 and z<=1:
        return 1-abs(z)
    else:
        return 0   

In [17]:
def tanh(z):
    """
    return the tanh of z
    """  
    return np.tanh(z)

In [18]:
def relu(z):
    """
    return the relu of z
    """ 
    return np.maximum(z, 0)

In [19]:
def layer_sizes(X, Y, hidden_layer):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    n_x = X.shape[0] # size of input layer
    n_h = hidden_layer
    n_y = Y.shape[0] # size of output layer
    
    return (n_x, n_h, n_y)

In [20]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    W1 = np.random.randn(n_h,n_x)*0.01
#     W1 = np.random.randn(n_h,n_x)
    b1 =  np.random.randn(n_h,1)
    

    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    
    parameters = {"W1": W1,
                  "b1": b1}
    
    return parameters

In [21]:
def activation_weights(X,parameters,activation):
    '''
    computes the activation weights i.e.the G matrix
    '''
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
#     print("X shape", X.shape)
#     print('W1 shape', W1.shape)
    
    Z = W1.dot(X) + b1
    if activation == 'sigmoid':
        G = sigmoid(Z)
    elif activation == 'tanh':
        G = tanh(Z)
    elif activation == 'rbf':
        G = rbf(Z)
    elif activation == 'relu':
        G = relu(Z)
    elif activation == 'triang_basis':
        triang_basis_vec = np.vectorize(triang_basis)
        G = triang_basis_vec(Z)        
    
#     print("G shape", G.shape)
    return G

def output_weights(G, Y):
    # computes the beta matrix
    
    output_weights = np.dot(Y,pinv2(G))
#     print('beta shape', output_weights.shape)
    return output_weights

In [22]:
def predict(X, parameters, output_weights, activation):
    '''
    predicts the closing values for n1 days starting from the i+1th day
    '''
    G_pred = activation_weights(X,parameters,activation)
    out = np.dot(output_weights,G_pred)
#     print(out.shape)
    return out

In [23]:
def scaled_close_value():
    mm_scaler = MinMaxScaler(feature_range=(-1,1))
    mm_scaler.fit(data.loc[:,['Close']])
    return mm_scaler

In [24]:
def complete_model(X, i, n, n1, n2, hidden_layer, activation, print_flag):
    '''
    Inputs:
    X: Normalized features
    i: prediction after ith day
    n: window size
    n1: future days to predict
    n2: number of history days to use to predict future stock price
    hidden_layer: # of hidden layers
    activation: type of activation function to use
    
    Outputs:
    RMSE on normalized data
    RMSE on actual data
    
    '''
    
    #training
    I, T = training_data_generation(X, i, n, n1, n2)
    n_x, n_h, n_y = layer_sizes(I, T, hidden_layer)
#     print(n_x,n_h,n_y)

    params = initialize_parameters(n_x, n_h, n_y)
    G_m = activation_weights(I,params,activation)
    beta = output_weights(G_m,T)
    
    # validation
    I_val, T_val = test_data_generation(X, i, n1, n2)
    pred = predict(I_val, params, beta, activation)
    
    rmse_with_scaling = mean_squared_error(T_val, pred)
    if print_flag:
        print('RMSE on scaled data: ', rmse_with_scaling)
    
    mm_cv_scaler = scaled_close_value()
    pred_denorm = mm_cv_scaler.inverse_transform(pred)
    act_denorm = mm_cv_scaler.inverse_transform(T_val)
    
    rmse_without_scaling = mean_squared_error(act_denorm, pred_denorm)
    if print_flag:
        print('RMSE on denormalized data: ', rmse_without_scaling)
        print('\n')
    Low_i = min(pred_denorm)[0]
    Up_i = max(pred_denorm)[0]
    
    return Low_i, Up_i
    
    

In [25]:
complete_model(X, 3000, 120, 15, 25, 16, 'tanh', True)

RMSE on scaled data:  0.0026995463293443774
RMSE on denormalized data:  21.100492813955007




(164.46123032956987, 173.797631360116)

In [26]:
data

,Date,Open,High,Low,Close,Volume,EWMA_15,ROC,RSI,FASTK,SLOWK,SLOWD,DOWN_K,UP_K
0,1/3/2006,6.999730,7.228929,6.987158,7.228929,201808600,NaN,NaN,NaN,NaN,NaN,NaN,7.193145,8.277245
1,1/4/2006,7.265678,7.347880,7.204753,7.250205,154900900,NaN,NaN,NaN,NaN,NaN,NaN,7.175739,8.277245
2,1/5/2006,7.236664,7.243433,7.132219,7.193145,112355600,NaN,NaN,NaN,NaN,NaN,NaN,6.994895,8.277245
3,1/6/2006,7.277282,7.417508,7.209586,7.378825,176114400,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245
4,1/9/2006,7.420411,7.465863,7.324670,7.354649,168760200,NaN,NaN,NaN,NaN,NaN,NaN,6.965883,8.277245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,4/27/2018,164.000000,164.330002,160.630005,162.320007,35655800,168.206091,-0.035990,39.886516,9.229941,7.237235,10.985416,NaN,NaN
3101,4/30/2018,162.130005,167.259995,161.839996,165.259995,42427400,167.837829,-0.028168,42.331075,25.286678,24.475479,16.214245,NaN,NaN
3102,5/1/2018,166.410004,169.199997,165.270004,169.100006,53569400,167.995601,-0.023954,43.489183,46.258888,46.258888,25.990534,NaN,NaN
3103,5/2/2018,175.229996,177.750000,173.800003,176.570007,66539400,169.067402,0.023950,55.359467,87.056279,87.056279,52.596882,NaN,NaN


In [27]:
from ta.trend import ADXIndicator

In [28]:
adxI = ADXIndicator(data['High'],data['Low'],data['Close'],14,False)
data['pos_directional_indicator'] = adxI.adx_pos()
data['neg_directional_indicator'] = adxI.adx_neg()
data['adx'] = adxI.adx()
data.tail(15)

,Date,Open,High,Low,Close,Volume,EWMA_15,ROC,RSI,FASTK,SLOWK,SLOWD,DOWN_K,UP_K,pos_directional_indicator,neg_directional_indicator,adx
3090,4/13/2018,174.779999,175.839996,173.850006,174.729996,25124300,171.922705,0.034824,57.550061,90.237463,94.042172,82.183021,162.320007,178.240005,22.785879,20.809239,11.013911
3091,4/16/2018,175.029999,176.190002,174.830002,175.820007,21578400,172.409867,0.065963,65.060905,96.843046,100.829224,93.660016,NaN,NaN,22.818969,20.229157,10.656924
3092,4/17/2018,176.490005,178.940002,176.410004,178.240005,26605400,173.138635,0.031661,58.905889,95.162426,109.984066,101.618488,NaN,NaN,26.756951,19.009632,11.104850
3093,4/18/2018,177.809998,178.820007,176.880005,177.839996,20754500,173.726305,0.056433,67.803584,92.398024,98.647275,103.153522,NaN,NaN,25.718717,18.272013,11.520782
3094,4/19/2018,173.759995,175.389999,172.660004,172.800003,34808800,173.610517,0.037963,60.582726,57.567391,57.567391,88.732911,NaN,NaN,23.137168,24.615322,10.918972
3095,4/20/2018,170.600006,171.220001,165.429993,165.720001,65491100,172.624203,-0.012278,47.109993,8.638562,8.638562,54.951076,NaN,NaN,20.053025,34.410756,12.022044
3096,4/23/2018,166.830002,166.919998,164.089996,165.240005,36515500,171.701178,-0.008639,47.944049,7.744165,6.142993,24.116315,NaN,NaN,19.005402,35.086719,13.286862
3097,4/24/2018,165.669998,166.330002,161.220001,162.940002,33692000,170.606031,-0.032365,42.347660,9.706551,-2.040799,4.246919,NaN,NaN,17.252727,37.030489,14.940258
3098,4/25/2018,162.619995,165.419998,162.410004,163.649994,28382100,169.736526,-0.046384,37.975817,13.713278,8.788990,4.297061,NaN,NaN,16.299227,34.983938,16.475553
3099,4/26/2018,164.119995,165.729996,163.369995,164.220001,27963000,169.046961,-0.049653,36.791867,16.930022,16.930022,7.892738,NaN,NaN,16.158173,33.424187,17.786078


# Box Theory

In [29]:
def trading_strategy(Ci, Lowi, Upi, prev_Lowi, prev_Upi, trans_rate_sigma, stop_loss_rate_theta, rho, phi, next_trade, buyprice,sellprice, flag):
    if next_trade == 'buy':
        if (abs(Ci - Lowi)/Ci <= trans_rate_sigma) and (Lowi > prev_Lowi):
            if flag==0:
                if (sellprice - Ci) >= rho:
                    buyprice = Ci
                    next_trade = 'sell'
            else:
                buyprice = Ci
                next_trade = 'sell'
                flag = 0
    else :
        if (abs(Ci - Upi)/Ci <= trans_rate_sigma) and (Upi < prev_Upi):
            if (Ci - buyprice) >= phi:
                    sellprice = Ci
                    next_trade = 'buy'
    return next_trade, buyprice, sellprice
        
    

In [30]:
def closing_vals(start_day, n1):
    return np.asarray(data.loc[start_day+1:start_day+n1,['Close']])

In [33]:
def trading(init_money, start_day, finish_day, trans_rate_sigma, stop_loss_rate_theta, rho, phi, next_trade_prev):
    money = init_money
    buyprice = 0
    sellprice = 0
    num_stocks = 0
    trans_cost = 1-0.005
    n1 = 15
    next_trade_prev = 'buy'
    flag = 1
    
    while(start_day <= finish_day):
#         print(start_day)
        prev_Lowi = data.at[start_day-n1+1,'DOWN_K']
        prev_Upi = data.at[start_day-n1+1,'UP_K']
        
        closing_values = closing_vals(start_day, n1)
        Lowi, Upi = complete_model(X, start_day, 90, 15, 25, 16, 'tanh', False)

        for cv in closing_values:
            next_trade_cur, buyprice, sellprice = trading_strategy(cv[0], Lowi, Upi, prev_Lowi, prev_Upi, trans_rate_sigma,+
                                              stop_loss_rate_theta, rho, phi, next_trade_prev, buyprice, sellprice, flag)
            
            
            if next_trade_cur != next_trade_prev:
                
                if next_trade_cur == 'sell':
                    print('Buyprice:',buyprice)
                    num_stocks = (money*trans_cost)//buyprice
                    print('Stock Bought')
                    print('Num stocks bought:', num_stocks)
                    print('\n')
                    
                elif next_trade_cur == 'buy':
                    print('Sellprice:',sellprice)
                    money = num_stocks*cv[0]*trans_cost
                    print('Stock Sold')
                    print('Money after selling the stock:', money)
                    print('\n')
                    
                next_trade_prev = next_trade_cur

        start_day = start_day + n1
    rate_of_profit = ((money - init_money)/init_money)*100
    print('Money after the end of trading is: $'+str(money))
    print('Rate of profit at the end of trading: '+ str(rate_of_profit) + '%')
    return money
    

In [34]:
trading(init_money = 10000, start_day = 400, finish_day = 1200, trans_rate_sigma=0.01, stop_loss_rate_theta = 0.1, rho = 0, phi=0.05, next_trade_prev = 'buy')

Buyprice: 12.844766
Stock Bought
Num stocks bought: 774.0


Sellprice: 17.232416
Stock Sold
Money after selling the stock: 13271.20053408


Buyprice: 8.574139
Stock Bought
Num stocks bought: 1540.0


Sellprice: 18.831961
Stock Sold
Money after selling the stock: 28856.2138403


Buyprice: 18.77297
Stock Bought
Num stocks bought: 1529.0


Sellprice: 23.814365
Stock Sold
Money after selling the stock: 36230.103264574995


Buyprice: 25.323013
Stock Bought
Num stocks bought: 1423.0


Money after the end of trading is: $36230.103264574995
Rate of profit at the end of trading: 262.30103264574996%


36230.103264574995